<a href="https://colab.research.google.com/github/mynameislllyt/API_Experiment/blob/main/baseline4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ============ 1. 读数据 + 拆 benign-only ============
def load_csv_expanded(path):
    df = pd.read_csv(path)
    seq_cols = [c for c in df.columns if c.startswith("t_")]
    seqs = df[seq_cols].values.astype(int)      # shape: [N, 100]
    labels = df["malware"].values.astype(int)   # 1=malware, 0=benign
    return seqs, labels

def split_benign_only(seqs, labels, seed=42):
    benign = seqs[labels == 0]
    malware = seqs[labels == 1]

    rng = np.random.default_rng(seed)
    idx_b = rng.permutation(len(benign))
    idx_m = rng.permutation(len(malware))

    n_b = len(benign)
    n_m = len(malware)

    n_train = int(0.7*n_b)
    n_val   = int(0.1*n_b)

    benign_train = benign[idx_b[:n_train]]
    benign_val   = benign[idx_b[n_train:n_train+n_val]]
    benign_test  = benign[idx_b[n_train+n_val:]]

    # 比如 20% malware 做 val，用来选阈值，剩下做 test
    n_m_val = int(0.2*n_m)
    malware_val  = malware[idx_m[:n_m_val]]
    malware_test = malware[idx_m[n_m_val:]]

    return benign_train, benign_val, benign_test, malware_val, malware_test

# ============ 2. 滑动窗口 ============
def make_windows(seqs, window_size=10):
    X, y = [], []
    for s in seqs:
        s = s.tolist()
        for i in range(len(s) - window_size):
            X.append(s[i:i+window_size])
            y.append(s[i+window_size])
    return np.array(X, dtype=int), np.array(y, dtype=int)

# ============ 3. Dataset ============
class WindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

# ============ 4. LSTM LM ============
class LSTMLM(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, hidden_dim=256, num_layers=1, dropout=0.2):
        super().__init__()
        # self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.emb = nn.Embedding(vocab_size, emb_dim)  # 去掉 padding_idx
        self.lstm = nn.LSTM(
            emb_dim, hidden_dim, num_layers=num_layers, batch_first=True,
            dropout=dropout if num_layers>1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        e = self.emb(x)        # [B, W, E]
        o, _ = self.lstm(e)    # [B, W, H]
        last = o[:, -1, :]
        return self.fc(last)   # [B, V]

# ============ 【新增】4.5. 记忆增强LSTM ============
class MemoryAugmentedLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, hidden_dim=256, num_layers=2,
                 dropout=0.3, memory_size=1000):
        super().__init__()
        # 复用原有的LSTM LM结构
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(
            emb_dim, hidden_dim, num_layers=num_layers, batch_first=True,
            dropout=dropout if num_layers>1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)

        # 新增：记忆库
        self.register_buffer('memory_bank', torch.zeros(memory_size, hidden_dim))
        self.memory_ptr = 0
        self.memory_size = memory_size
        self.hidden_dim = hidden_dim

    def forward(self, x, return_hidden=False):
        """前向传播，可选返回hidden state"""
        e = self.emb(x)        # [B, W, E]
        o, (h, c) = self.lstm(e)    # h: [num_layers, B, H]
        logits = self.fc(o[:, -1, :])   # [B, V]

        if return_hidden:
            return logits, h[-1]  # 返回最后一层的hidden
        return logits

    def update_memory(self, hidden_states):
        """训练时更新记忆库"""
        with torch.no_grad():
            batch_size = hidden_states.size(0)
            ptr = self.memory_ptr

            if ptr + batch_size <= self.memory_size:
                self.memory_bank[ptr:ptr+batch_size] = hidden_states.detach()
                self.memory_ptr = (ptr + batch_size) % self.memory_size
            else:
                remain = self.memory_size - ptr
                self.memory_bank[ptr:] = hidden_states[:remain].detach()
                self.memory_bank[:batch_size-remain] = hidden_states[remain:].detach()
                self.memory_ptr = batch_size - remain

    def compute_memory_distance(self, hidden_state):
        """计算与记忆库的距离"""
        # hidden_state: [B, H]
        distances = torch.cdist(hidden_state, self.memory_bank)  # [B, M]
        k = min(50, self.memory_size)
        min_distances, _ = torch.topk(distances, k, largest=False, dim=1)
        return min_distances.mean(dim=1)  # [B]

# ============ 5. 训练/验证 ============
def eval_loss(model, loader, device="cuda"):
    model.eval()
    crit = nn.CrossEntropyLoss(reduction="sum")
    total = 0.0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            total += crit(model(Xb), yb).item()
    return total / len(loader.dataset)

def train_model(model, train_loader, val_loader, epochs=20, lr=1e-3, device="cuda"):
    model.to(device)
    #opt = torch.optim.Adam(model.parameters(), lr=lr)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

    crit = nn.CrossEntropyLoss()

    best_val, best_state = 1e9, None
    patience, bad_count = 5, 0
    for ep in range(1, epochs+1):
        model.train()
        total = 0.0
        for Xb, yb in tqdm(train_loader, desc=f"Epoch {ep}"):
            Xb, yb = Xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = crit(model(Xb), yb)
            loss.backward()
            opt.step()
            total += loss.item() * Xb.size(0)

        val_loss = eval_loss(model, val_loader, device)
        print(f"ep{ep}: train={total/len(train_loader.dataset):.4f}, val={val_loss:.4f}")
        # if val_loss < best_val:
        #     best_val = val_loss
        #     best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
        if val_loss < best_val - 1e-4:
            best_val = val_loss
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
            bad_count = 0
        else:
            bad_count += 1
            if bad_count >= patience:
                print(f"Early stop at epoch {ep}")
                break

    model.load_state_dict(best_state)
    return model

# ============ 【修改】5.5. 训练记忆增强模型 ============
def train_model_with_memory(model, train_loader, val_loader, epochs=20, lr=1e-3, device="cuda"):
    """训练记忆增强模型，在训练过程中更新记忆库"""
    """
    在 LSTMLM 的基础上加一个“隐状态记忆库 (memory_bank)”：
      - 训练时，把 benign 的 hidden state 持续写入 memory_bank
      - 检测时，根据当前样本 hidden 与记忆库中向量的距离，衡量“是否偏离正常模式”
      - 最终异常分数 = 语言模型 NLL + 记忆距离 的组合
    """
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    crit = nn.CrossEntropyLoss()

    best_val, best_state = 1e9, None
    patience, bad_count = 5, 0

    for ep in range(1, epochs+1):
        model.train()
        total = 0.0
        for Xb, yb in tqdm(train_loader, desc=f"Epoch {ep}"):
            Xb, yb = Xb.to(device), yb.to(device)
            opt.zero_grad()

            # 获取logits和hidden states
            logits, hidden = model(Xb, return_hidden=True)
            loss = crit(logits, yb)
            loss.backward()
            opt.step()
            total += loss.item() * Xb.size(0)

            # 【关键】更新记忆库
            model.update_memory(hidden)

        val_loss = eval_loss_memory(model, val_loader, device)
        print(f"ep{ep}: train={total/len(train_loader.dataset):.4f}, val={val_loss:.4f}")

        if val_loss < best_val - 1e-4:
            best_val = val_loss
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
            bad_count = 0
        else:
            bad_count += 1
            if bad_count >= patience:
                print(f"Early stop at epoch {ep}")
                break

    model.load_state_dict(best_state)
    return model

def eval_loss_memory(model, loader, device="cuda"):
    """验证记忆增强模型"""
    model.eval()
    crit = nn.CrossEntropyLoss(reduction="sum")
    total = 0.0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            logits = model(Xb, return_hidden=False)
            total += crit(logits, yb).item()
    return total / len(loader.dataset)

# ============ 6. 序列 NLL 异常分数 ============
def sequence_scores_nll(model, seqs, window_size=10, device="cuda"):#score 越大 = 越不符合 benign 模式 = 越可疑
    crit = nn.CrossEntropyLoss(reduction="none")
    model.eval()
    scores = []

    with torch.no_grad():
        for s in seqs:
            X, y = make_windows([s], window_size)
            X = torch.tensor(X, dtype=torch.long).to(device)
            y = torch.tensor(y, dtype=torch.long).to(device)
            nll = crit(model(X), y)
            scores.append(nll.mean().item())
    return np.array(scores)

def search_best_threshold(benign_scores, malware_scores):
    all_scores = np.concatenate([benign_scores, malware_scores])
    cand_th = np.quantile(all_scores, np.linspace(0.7, 0.99, 20))  # 可以调范围

    best_f1, best_th, best_metrics = -1, None, None
    for th in cand_th:
        m = evaluate(th, benign_scores, malware_scores)
        if m["f1"] > best_f1:
            best_f1, best_th, best_metrics = m["f1"], th, m
    return best_th, best_metrics

def pick_threshold(val_scores, q=0.99):#在验证集上，大约 99% 的 benign score 都 低于 这个阈值
    return float(np.quantile(val_scores, q))

def evaluate(th, benign_scores, malware_scores):
    y_true = np.array([0]*len(benign_scores) + [1]*len(malware_scores))
    y_pred = np.array(
        [1 if s>th else 0 for s in benign_scores] +
        [1 if s>th else 0 for s in malware_scores]
    )
    tp = ((y_true==1)&(y_pred==1)).sum()
    tn = ((y_true==0)&(y_pred==0)).sum()
    fp = ((y_true==0)&(y_pred==1)).sum()
    fn = ((y_true==1)&(y_pred==0)).sum()

    precision = tp/(tp+fp+1e-9)
    recall    = tp/(tp+fn+1e-9)
    f1        = 2*precision*recall/(precision+recall+1e-9)
    acc       = (tp+tn)/(tp+tn+fp+fn+1e-9)
    fpr       = fp/(fp+tn+1e-9)
    return dict(acc=acc, precision=precision, recall=recall, f1=f1, fpr=fpr,
                tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))

# ============ 【新增】6.5. 混合异常分数（NLL + Memory Distance）============
def sequence_scores_hybrid(model, seqs, window_size=10, alpha=0.5, device="cuda"):
    """计算混合异常分数：NLL + 记忆距离"""
    """
    对每条序列同时计算：
      - NLL 分数（语言模型下一个 token 的平均负对数似然）
      - Memory Distance 分数（hidden state 与记忆库中向量的距离）

    然后做标准化 + 加权求和：
      hybrid_score = alpha * NLL_norm + (1-alpha) * Mem_norm
    """
    crit = nn.CrossEntropyLoss(reduction="none")
    model.eval()
    nll_scores = []
    mem_scores = []

    with torch.no_grad():
        for s in seqs:
            X, y = make_windows([s], window_size)
            if len(X) == 0:  # 处理序列太短的情况
                nll_scores.append(0.0)
                mem_scores.append(0.0)
                continue

            X = torch.tensor(X, dtype=torch.long).to(device)
            y = torch.tensor(y, dtype=torch.long).to(device)

            # 1. NLL分数
            logits, hidden = model(X, return_hidden=True)
            nll = crit(logits, y).mean().item()
            nll_scores.append(nll)

            # 2. 记忆距离分数
            mem_dist = model.compute_memory_distance(hidden).mean().item()
            mem_scores.append(mem_dist)

    # 归一化后加权融合
    nll_scores = np.array(nll_scores)
    mem_scores = np.array(mem_scores)

    nll_norm = (nll_scores - nll_scores.mean()) / (nll_scores.std() + 1e-8)
    mem_norm = (mem_scores - mem_scores.mean()) / (mem_scores.std() + 1e-8)

    hybrid_scores = alpha * nll_norm + (1 - alpha) * mem_norm
    return hybrid_scores

# ============ 【新增】7. 多窗口集成 ============
class MultiWindowEnsemble:
  """
    多窗口集成器：
      - 为多个 window_size 各自训练一个模型（可以是普通 LSTM 或 Memory-Augmented LSTM）
      - 对同一条序列，用多个窗口尺度分别打分+判决
      - 通过投票 / 最大分数等策略做集成，提升鲁棒性
    """
    def __init__(self, window_sizes=[10, 20, 30], vocab_size=307, use_memory=False):
        self.window_sizes = window_sizes
        self.vocab_size = vocab_size
        self.use_memory = use_memory
        self.models = {}
        self.thresholds = {}

    def train_all(self, benign_train, benign_val, device="cuda"):
        """为每个窗口大小训练独立模型"""
        for ws in self.window_sizes:
            print(f"\n{'='*60}")
            print(f"Training window_size={ws}")

            Xtr, ytr = make_windows(benign_train, ws)
            Xva, yva = make_windows(benign_val, ws)

            train_loader = DataLoader(WindowDataset(Xtr, ytr),
                                     batch_size=256, shuffle=True)
            val_loader = DataLoader(WindowDataset(Xva, yva), batch_size=256)

            # 根据配置选择模型
            if self.use_memory:
                model = MemoryAugmentedLSTM(self.vocab_size, emb_dim=128,
                                           hidden_dim=256, num_layers=2, dropout=0.3)
                model = train_model_with_memory(model, train_loader, val_loader, device=device)
            else:
                model = LSTMLM(self.vocab_size, emb_dim=128,
                              hidden_dim=256, num_layers=2, dropout=0.3)
                model = train_model(model, train_loader, val_loader, device=device)

            self.models[ws] = model

            # 在验证集上确定阈值
            if self.use_memory:
                val_scores = sequence_scores_hybrid(model, benign_val, ws, alpha=0.6, device=device)
            else:
                val_scores = sequence_scores_nll(model, benign_val, ws, device)
            self.thresholds[ws] = np.quantile(val_scores, 0.99)

    def predict_ensemble(self, seqs, device="cuda", method='vote'):
        """集成预测"""
        """
        对一批序列做集成预测：
          - method='vote'：各模型独立二分类，最后按多数投票
          - method='max'：归一化各模型分数，取每个样本在各模型中的最大分数再判断
        返回：
          - final_pred: 最终的 0/1 预测
          - all_scores: 每个窗口大小对应的原始分数字典
        """
        all_predictions = {}
        all_scores = {}

        # 每个模型独立预测
        for ws in self.window_sizes:
            if self.use_memory:
                scores = sequence_scores_hybrid(self.models[ws], seqs, ws,
                                               alpha=0.6, device=device)
            else:
                scores = sequence_scores_nll(self.models[ws], seqs, ws, device)

            predictions = (scores > self.thresholds[ws]).astype(int)
            all_predictions[ws] = predictions
            all_scores[ws] = scores

        # 集成策略
        if method == 'vote':
            # 投票法
            stacked = np.stack([all_predictions[ws] for ws in self.window_sizes])
            final_pred = (stacked.sum(axis=0) > len(self.window_sizes) / 2).astype(int)

        elif method == 'max':
            # 最大分数法
            normalized_scores = []
            for ws in self.window_sizes:
                scores = all_scores[ws]
                norm_scores = (scores - scores.mean()) / (scores.std() + 1e-8)
                normalized_scores.append(norm_scores)

            max_scores = np.max(normalized_scores, axis=0)
            threshold = 0  # 已归一化，0为中值
            final_pred = (max_scores > threshold).astype(int)

        return final_pred, all_scores
# ============ 7. 主流程 ============
def main():
    path = "./dynamic_api_call_sequence_per_malware_100_0_306.csv"
    seqs, labels = load_csv_expanded(path)
    benign_train, benign_val, benign_test, malware_val, malware_test = split_benign_only(seqs, labels)

    vocab_size = int(seqs.max()) + 1
    print("vocab_size:", vocab_size)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # ========== 实验1：原始Baseline（保留用于对比）==========
    print("\n" + "="*70)
    print("实验1: 原始Baseline - 单窗口LSTM")
    print("="*70)

    window_size = 20  # 选择表现最好的窗口
    Xtr, ytr = make_windows(benign_train, window_size)
    Xva, yva = make_windows(benign_val, window_size)

    train_loader = DataLoader(WindowDataset(Xtr, ytr), batch_size=256, shuffle=True)
    val_loader = DataLoader(WindowDataset(Xva, yva), batch_size=256)

    model_baseline = LSTMLM(vocab_size, emb_dim=128, hidden_dim=256, num_layers=2, dropout=0.3)
    model_baseline = train_model(model_baseline, train_loader, val_loader, device=device)

    benign_val_scores = sequence_scores_nll(model_baseline, benign_val, window_size, device=device)
    malware_val_scores = sequence_scores_nll(model_baseline, malware_val, window_size, device=device)
    best_th_baseline, _ = search_best_threshold(benign_val_scores, malware_val_scores)

    benign_test_scores = sequence_scores_nll(model_baseline, benign_test, window_size, device=device)
    malware_test_scores = sequence_scores_nll(model_baseline, malware_test, window_size, device=device)
    metrics_baseline = evaluate(best_th_baseline, benign_test_scores, malware_test_scores)

    print(f"\n【Baseline结果】")
    print(f"Threshold: {best_th_baseline:.4f}")
    print(f"Test Metrics: {metrics_baseline}")

    # ========== 实验2：记忆增强LSTM ==========
    print("\n" + "="*70)
    print("实验2: 记忆增强LSTM (创新点1)")
    print("="*70)

    model_memory = MemoryAugmentedLSTM(vocab_size, emb_dim=128, hidden_dim=256,
                                       num_layers=2, dropout=0.3, memory_size=1000)
    model_memory = train_model_with_memory(model_memory, train_loader, val_loader, device=device)

    # 测试不同alpha值
    for alpha in [0.3, 0.5, 0.7]:
        print(f"\n--- Alpha = {alpha} ---")
        benign_val_hybrid = sequence_scores_hybrid(model_memory, benign_val, window_size,
                                                   alpha=alpha, device=device)
        malware_val_hybrid = sequence_scores_hybrid(model_memory, malware_val, window_size,
                                                    alpha=alpha, device=device)
        best_th_hybrid, _ = search_best_threshold(benign_val_hybrid, malware_val_hybrid)

        benign_test_hybrid = sequence_scores_hybrid(model_memory, benign_test, window_size,
                                                    alpha=alpha, device=device)
        malware_test_hybrid = sequence_scores_hybrid(model_memory, malware_test, window_size,
                                                     alpha=alpha, device=device)
        metrics_hybrid = evaluate(best_th_hybrid, benign_test_hybrid, malware_test_hybrid)

        print(f"Threshold: {best_th_hybrid:.4f}")
        print(f"Test Metrics: {metrics_hybrid}")

    # ========== 实验3：多窗口集成（不带记忆）==========
    print("\n" + "="*70)
    print("实验3: 多窗口集成 (创新点2)")
    print("="*70)

    ensemble_basic = MultiWindowEnsemble(window_sizes=[10, 20, 30],
                                         vocab_size=vocab_size, use_memory=False)
    ensemble_basic.train_all(benign_train, benign_val, device)

    test_seqs = np.concatenate([benign_test, malware_test])
    y_true = np.array([0]*len(benign_test) + [1]*len(malware_test))

    for method in ['vote', 'max']:
        print(f"\n--- Ensemble Method: {method} ---")
        pred_test, _ = ensemble_basic.predict_ensemble(test_seqs, device, method=method)

        tp = ((y_true==1)&(pred_test==1)).sum()
        tn = ((y_true==0)&(pred_test==0)).sum()
        fp = ((y_true==0)&(pred_test==1)).sum()
        fn = ((y_true==1)&(pred_test==0)).sum()

        precision = tp/(tp+fp+1e-9)
        recall = tp/(tp+fn+1e-9)
        f1 = 2*precision*recall/(precision+recall+1e-9)
        acc = (tp+tn)/(tp+tn+fp+fn+1e-9)

        metrics_ensemble = dict(acc=acc, precision=precision, recall=recall, f1=f1,
                               tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))
        print(f"Test Metrics: {metrics_ensemble}")

    # ========== 实验4：记忆增强 + 多窗口集成 ==========
    print("\n" + "="*70)
    print("实验4: 记忆增强多窗口集成 (组合创新)")
    print("="*70)

    ensemble_memory = MultiWindowEnsemble(window_sizes=[10, 20, 30],
                                          vocab_size=vocab_size, use_memory=True)
    ensemble_memory.train_all(benign_train, benign_val, device)

    for method in ['vote', 'max']:
        print(f"\n--- Ensemble Method: {method} ---")
        pred_test, _ = ensemble_memory.predict_ensemble(test_seqs, device, method=method)

        tp = ((y_true==1)&(pred_test==1)).sum()
        tn = ((y_true==0)&(pred_test==0)).sum()
        fp = ((y_true==0)&(pred_test==1)).sum()
        fn = ((y_true==1)&(pred_test==0)).sum()

        precision = tp/(tp+fp+1e-9)
        recall = tp/(tp+fn+1e-9)
        f1 = 2*precision*recall/(precision+recall+1e-9)
        acc = (tp+tn)/(tp+tn+fp+fn+1e-9)

        metrics_final = dict(acc=acc, precision=precision, recall=recall, f1=f1,
                            tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))
        print(f"Test Metrics: {metrics_final}")



if __name__ == "__main__":
    main()

vocab_size: 307

实验1: 原始Baseline - 单窗口LSTM


Epoch 1: 100%|██████████| 236/236 [00:03<00:00, 62.42it/s]


ep1: train=2.2689, val=1.4963


Epoch 2: 100%|██████████| 236/236 [00:02<00:00, 86.55it/s]


ep2: train=1.1903, val=1.1313


...

ep13: train=0.2998, val=0.7913


Epoch 14: 100%|██████████| 207/207 [00:03<00:00, 57.47it/s]


ep14: train=0.2752, val=0.7938


Epoch 15: 100%|██████████| 207/207 [00:03<00:00, 58.97it/s]


ep15: train=0.2536, val=0.8101
Early stop at epoch 15

--- Ensemble Method: vote ---
Test Metrics: {'acc': np.float64(0.018778116383688325), 'precision': np.float64(0.995391705066831), 'recall': np.float64(0.012617559436882628), 'f1': np.float64(0.024919243168632185), 'tp': 432, 'tn': 215, 'fp': 2, 'fn': 33806}

--- Ensemble Method: max ---
Test Metrics: {'acc': np.float64(0.5046872732549557), 'precision': np.float64(0.996645071725995), 'recall': np.float64(0.50324201179974), 'f1': np.float64(0.6687885723134651), 'tp': 17230, 'tn': 159, 'fp': 58, 'fn': 17008}


In [2]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy

# ============ 1. 读数据 + 拆 benign-only ============
def load_csv_expanded(path):
    df = pd.read_csv(path)
    seq_cols = [c for c in df.columns if c.startswith("t_")]
    seqs = df[seq_cols].values.astype(int)
    labels = df["malware"].values.astype(int)
    return seqs, labels

def split_benign_only(seqs, labels, seed=42):
    benign = seqs[labels == 0]
    malware = seqs[labels == 1]

    rng = np.random.default_rng(seed)
    idx_b = rng.permutation(len(benign))
    idx_m = rng.permutation(len(malware))

    n_b = len(benign)
    n_m = len(malware)

    n_train = int(0.7*n_b)
    n_val   = int(0.1*n_b)

    benign_train = benign[idx_b[:n_train]]
    benign_val   = benign[idx_b[n_train:n_train+n_val]]
    benign_test  = benign[idx_b[n_train+n_val:]]

    n_m_val = int(0.2*n_m)
    malware_val  = malware[idx_m[:n_m_val]]
    malware_test = malware[idx_m[n_m_val:]]

    return benign_train, benign_val, benign_test, malware_val, malware_test

# ============ 2. 滑动窗口 ============
def make_windows(seqs, window_size=10):
    X, y = [], []
    for s in seqs:
        s = s.tolist()
        for i in range(len(s) - window_size):
            X.append(s[i:i+window_size])
            y.append(s[i+window_size])
    return np.array(X, dtype=int), np.array(y, dtype=int)

# ============ 3. Dataset ============
class WindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

# ============ 4. LSTM LM ============
class LSTMLM(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, hidden_dim=256, num_layers=2, dropout=0.3):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(
            emb_dim, hidden_dim, num_layers=num_layers, batch_first=True,
            dropout=dropout if num_layers>1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        e = self.emb(x)
        o, _ = self.lstm(e)
        last = o[:, -1, :]
        return self.fc(last)

# ============ 【改进】4.5. 增强版记忆增强LSTM ============
class ImprovedMemoryAugmentedLSTM(nn.Module):
    """改进版记忆增强LSTM：使用余弦相似度 + 更大记忆库"""
    def __init__(self, vocab_size, emb_dim=128, hidden_dim=256, num_layers=2,
                 dropout=0.3, memory_size=5000):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(
            emb_dim, hidden_dim, num_layers=num_layers, batch_first=True,
            dropout=dropout if num_layers>1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)

        self.register_buffer('memory_bank', torch.zeros(memory_size, hidden_dim))
        self.memory_ptr = 0
        self.memory_size = memory_size
        self.hidden_dim = hidden_dim

    def forward(self, x, return_hidden=False):
        e = self.emb(x)
        o, (h, c) = self.lstm(e)
        logits = self.fc(o[:, -1, :])

        if return_hidden:
            return logits, h[-1]
        return logits

    def update_memory(self, hidden_states):
        with torch.no_grad():
            batch_size = hidden_states.size(0)
            ptr = self.memory_ptr

            if ptr + batch_size <= self.memory_size:
                self.memory_bank[ptr:ptr+batch_size] = hidden_states.detach()
                self.memory_ptr = (ptr + batch_size) % self.memory_size
            else:
                remain = self.memory_size - ptr
                self.memory_bank[ptr:] = hidden_states[:remain].detach()
                self.memory_bank[:batch_size-remain] = hidden_states[remain:].detach()
                self.memory_ptr = batch_size - remain

    def compute_memory_distance(self, hidden_state):
        """【改进】使用余弦相似度代替欧氏距离"""
        hidden_norm = F.normalize(hidden_state, dim=1)
        memory_norm = F.normalize(self.memory_bank, dim=1)

        similarity = torch.mm(hidden_norm, memory_norm.T)
        distances = 1 - similarity

        k = min(100, self.memory_size)
        min_distances, _ = torch.topk(distances, k, largest=False, dim=1)
        return min_distances.mean(dim=1)

# ============ 【新增】4.6 特征工程 ============
def extract_statistical_features(seq):
    """提取序列的统计特征"""
    seq = seq[seq > 0]
    if len(seq) == 0:
        return np.zeros(6)

    features = []

    # 1. API多样性
    unique_ratio = len(set(seq)) / len(seq)
    features.append(unique_ratio)

    # 2. 序列熵
    counts = np.bincount(seq)
    probs = counts[counts > 0] / len(seq)
    seq_entropy = entropy(probs)
    features.append(seq_entropy)

    # 3. 最长连续重复
    max_repeat = 1
    current_repeat = 1
    for i in range(1, len(seq)):
        if seq[i] == seq[i-1]:
            current_repeat += 1
            max_repeat = max(max_repeat, current_repeat)
        else:
            current_repeat = 1
    features.append(max_repeat / len(seq))

    # 4. API调用频率的标准差
    unique_apis = list(set(seq))
    api_freq = [list(seq).count(api) for api in unique_apis]
    features.append(np.std(api_freq) if len(api_freq) > 0 else 0)

    # 5. 罕见API比例
    rare_count = sum(1 for v in counts if 0 < v < 3)
    features.append(rare_count / len(set(seq)) if len(set(seq)) > 0 else 0)

    # 6. 平均API转移距离
    transitions = np.abs(np.diff(seq))
    features.append(np.mean(transitions) if len(transitions) > 0 else 0)

    return np.array(features)

def extract_features_batch(seqs):
    """批量提取特征"""
    return np.array([extract_statistical_features(seq) for seq in seqs])

# ============ 5. 训练/验证 ============
def eval_loss(model, loader, device="cuda"):
    model.eval()
    crit = nn.CrossEntropyLoss(reduction="sum")
    total = 0.0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            total += crit(model(Xb), yb).item()
    return total / len(loader.dataset)

def train_model(model, train_loader, val_loader, epochs=20, lr=1e-3, device="cuda"):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    crit = nn.CrossEntropyLoss()

    best_val, best_state = 1e9, None
    patience, bad_count = 5, 0
    for ep in range(1, epochs+1):
        model.train()
        total = 0.0
        for Xb, yb in tqdm(train_loader, desc=f"Epoch {ep}", leave=False):
            Xb, yb = Xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = crit(model(Xb), yb)
            loss.backward()
            opt.step()
            total += loss.item() * Xb.size(0)

        val_loss = eval_loss(model, val_loader, device)
        if ep % 5 == 0 or ep == 1:
            print(f"ep{ep}: train={total/len(train_loader.dataset):.4f}, val={val_loss:.4f}")

        if val_loss < best_val - 1e-4:
            best_val = val_loss
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
            bad_count = 0
        else:
            bad_count += 1
            if bad_count >= patience:
                print(f"Early stop at epoch {ep}")
                break

    model.load_state_dict(best_state)
    return model

def train_model_with_memory(model, train_loader, val_loader, epochs=20, lr=1e-3, device="cuda"):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    crit = nn.CrossEntropyLoss()

    best_val, best_state = 1e9, None
    patience, bad_count = 5, 0

    for ep in range(1, epochs+1):
        model.train()
        total = 0.0
        for Xb, yb in tqdm(train_loader, desc=f"Epoch {ep}", leave=False):
            Xb, yb = Xb.to(device), yb.to(device)
            opt.zero_grad()

            logits, hidden = model(Xb, return_hidden=True)
            loss = crit(logits, yb)
            loss.backward()
            opt.step()
            total += loss.item() * Xb.size(0)

            model.update_memory(hidden)

        val_loss = eval_loss_memory(model, val_loader, device)
        if ep % 5 == 0 or ep == 1:
            print(f"ep{ep}: train={total/len(train_loader.dataset):.4f}, val={val_loss:.4f}")

        if val_loss < best_val - 1e-4:
            best_val = val_loss
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
            bad_count = 0
        else:
            bad_count += 1
            if bad_count >= patience:
                print(f"Early stop at epoch {ep}")
                break

    model.load_state_dict(best_state)
    return model

def eval_loss_memory(model, loader, device="cuda"):
    model.eval()
    crit = nn.CrossEntropyLoss(reduction="sum")
    total = 0.0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            logits = model(Xb, return_hidden=False)
            total += crit(logits, yb).item()
    return total / len(loader.dataset)

# ============ 6. 序列异常分数 ============
def sequence_scores_nll(model, seqs, window_size=10, device="cuda"):
    crit = nn.CrossEntropyLoss(reduction="none")
    model.eval()
    scores = []

    with torch.no_grad():
        for s in seqs:
            X, y = make_windows([s], window_size)
            if len(X) == 0:
                scores.append(0.0)
                continue
            X = torch.tensor(X, dtype=torch.long).to(device)
            y = torch.tensor(y, dtype=torch.long).to(device)
            nll = crit(model(X), y)
            scores.append(nll.mean().item())
    return np.array(scores)

def sequence_scores_hybrid_improved(model, seqs, window_size=10, device="cuda"):
    """【改进】自适应加权的混合分数"""
    crit = nn.CrossEntropyLoss(reduction="none")
    model.eval()
    nll_scores = []
    mem_scores = []

    with torch.no_grad():
        for s in seqs:
            X, y = make_windows([s], window_size)
            if len(X) == 0:
                nll_scores.append(0.0)
                mem_scores.append(0.0)
                continue

            X = torch.tensor(X, dtype=torch.long).to(device)
            y = torch.tensor(y, dtype=torch.long).to(device)

            logits, hidden = model(X, return_hidden=True)
            nll = crit(logits, y).mean().item()
            nll_scores.append(nll)

            mem_dist = model.compute_memory_distance(hidden).mean().item()
            mem_scores.append(mem_dist)

    nll_scores = np.array(nll_scores)
    mem_scores = np.array(mem_scores)

    # 【改进】使用中位数+IQR归一化
    nll_median = np.median(nll_scores)
    nll_iqr = np.percentile(nll_scores, 75) - np.percentile(nll_scores, 25)
    nll_norm = (nll_scores - nll_median) / (nll_iqr + 1e-8)

    mem_median = np.median(mem_scores)
    mem_iqr = np.percentile(mem_scores, 75) - np.percentile(mem_scores, 25)
    mem_norm = (mem_scores - mem_median) / (mem_iqr + 1e-8)

    # 【改进】自适应权重
    nll_std = np.std(nll_norm)
    mem_std = np.std(mem_norm)
    alpha = nll_std / (nll_std + mem_std + 1e-8)

    hybrid_scores = alpha * nll_norm + (1 - alpha) * mem_norm
    return hybrid_scores, alpha

def search_best_threshold(benign_scores, malware_scores):
    all_scores = np.concatenate([benign_scores, malware_scores])
    cand_th = np.quantile(all_scores, np.linspace(0.7, 0.99, 30))

    best_f1, best_th, best_metrics = -1, None, None
    for th in cand_th:
        m = evaluate(th, benign_scores, malware_scores)
        if m["f1"] > best_f1:
            best_f1, best_th, best_metrics = m["f1"], th, m
    return best_th, best_metrics

def evaluate(th, benign_scores, malware_scores):
    y_true = np.array([0]*len(benign_scores) + [1]*len(malware_scores))
    y_pred = np.array(
        [1 if s>th else 0 for s in benign_scores] +
        [1 if s>th else 0 for s in malware_scores]
    )
    tp = ((y_true==1)&(y_pred==1)).sum()
    tn = ((y_true==0)&(y_pred==0)).sum()
    fp = ((y_true==0)&(y_pred==1)).sum()
    fn = ((y_true==1)&(y_pred==0)).sum()

    precision = tp/(tp+fp+1e-9)
    recall    = tp/(tp+fn+1e-9)
    f1        = 2*precision*recall/(precision+recall+1e-9)
    acc       = (tp+tn)/(tp+tn+fp+fn+1e-9)
    fpr       = fp/(fp+tn+1e-9)
    return dict(acc=acc, precision=precision, recall=recall, f1=f1, fpr=fpr,
                tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))

# ============ 7. 多窗口集成 ============
class ImprovedMultiWindowEnsemble:
    """改进版多窗口集成"""
    def __init__(self, window_sizes=[10, 20, 30], vocab_size=307, use_memory=False):
        self.window_sizes = window_sizes
        self.vocab_size = vocab_size
        self.use_memory = use_memory
        self.models = {}
        self.thresholds = {}
        self.weights = {}

    def train_all(self, benign_train, benign_val, malware_val, device="cuda"):
        """训练所有窗口的模型并计算权重"""
        val_f1_scores = {}

        for ws in self.window_sizes:
            print(f"\n{'='*60}")
            print(f"Training window_size={ws}")

            Xtr, ytr = make_windows(benign_train, ws)
            Xva, yva = make_windows(benign_val, ws)

            train_loader = DataLoader(WindowDataset(Xtr, ytr), batch_size=256, shuffle=True)
            val_loader = DataLoader(WindowDataset(Xva, yva), batch_size=256)

            if self.use_memory:
                model = ImprovedMemoryAugmentedLSTM(self.vocab_size, emb_dim=128,
                                           hidden_dim=256, num_layers=2, dropout=0.3,
                                           memory_size=5000)
                model = train_model_with_memory(model, train_loader, val_loader, device=device)
            else:
                model = LSTMLM(self.vocab_size, emb_dim=128,
                              hidden_dim=256, num_layers=2, dropout=0.3)
                model = train_model(model, train_loader, val_loader, device=device)

            self.models[ws] = model

            # 在验证集上评估并找最佳阈值
            if self.use_memory:
                benign_val_scores, _ = sequence_scores_hybrid_improved(model, benign_val, ws, device=device)
                malware_val_scores, _ = sequence_scores_hybrid_improved(model, malware_val, ws, device=device)
            else:
                benign_val_scores = sequence_scores_nll(model, benign_val, ws, device)
                malware_val_scores = sequence_scores_nll(model, malware_val, ws, device)

            best_th, metrics = search_best_threshold(benign_val_scores, malware_val_scores)
            self.thresholds[ws] = best_th
            val_f1_scores[ws] = metrics['f1']
            print(f"Val F1: {metrics['f1']:.4f}, Threshold: {best_th:.4f}")

        # 【改进】根据验证集F1计算权重
        total_f1 = sum(val_f1_scores.values())
        for ws in self.window_sizes:
            self.weights[ws] = val_f1_scores[ws] / total_f1
            print(f"Window {ws} weight: {self.weights[ws]:.4f}")

    def predict_ensemble(self, seqs, device="cuda", method='weighted'):
        """【改进】增加加权融合方法"""
        all_predictions = {}
        all_scores = {}

        for ws in self.window_sizes:
            if self.use_memory:
                scores, _ = sequence_scores_hybrid_improved(self.models[ws], seqs, ws, device=device)
            else:
                scores = sequence_scores_nll(self.models[ws], seqs, ws, device)

            predictions = (scores > self.thresholds[ws]).astype(int)
            all_predictions[ws] = predictions
            all_scores[ws] = scores

        if method == 'vote':
            stacked = np.stack([all_predictions[ws] for ws in self.window_sizes])
            final_pred = (stacked.sum(axis=0) > len(self.window_sizes) / 2).astype(int)

        elif method == 'max':
            normalized_scores = []
            for ws in self.window_sizes:
                scores = all_scores[ws]
                norm_scores = (scores - scores.mean()) / (scores.std() + 1e-8)
                normalized_scores.append(norm_scores)

            max_scores = np.max(normalized_scores, axis=0)
            threshold = 0
            final_pred = (max_scores > threshold).astype(int)

        elif method == 'weighted':
            normalized_scores = []
            for ws in self.window_sizes:
                scores = all_scores[ws]
                norm_scores = (scores - scores.mean()) / (scores.std() + 1e-8)
                normalized_scores.append(norm_scores * self.weights[ws])

            weighted_scores = np.sum(normalized_scores, axis=0)
            threshold = 0
            final_pred = (weighted_scores > threshold).astype(int)

        return final_pred, all_scores

# ============ 【新增】8. 传统方法对比 ============
def train_isolation_forest(benign_train, benign_test, malware_test):
    """Isolation Forest基准"""
    print("\n" + "="*70)
    print("训练 Isolation Forest")
    print("="*70)

    train_features = extract_features_batch(benign_train)
    test_benign_features = extract_features_batch(benign_test)
    test_malware_features = extract_features_batch(malware_test)

    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    test_benign_features = scaler.transform(test_benign_features)
    test_malware_features = scaler.transform(test_malware_features)

    clf = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
    clf.fit(train_features)

    pred_benign = clf.predict(test_benign_features)
    pred_malware = clf.predict(test_malware_features)

    pred_benign = (pred_benign == -1).astype(int)
    pred_malware = (pred_malware == -1).astype(int)

    y_true = np.array([0]*len(benign_test) + [1]*len(malware_test))
    y_pred = np.concatenate([pred_benign, pred_malware])

    tp = ((y_true==1)&(y_pred==1)).sum()
    tn = ((y_true==0)&(y_pred==0)).sum()
    fp = ((y_true==0)&(y_pred==1)).sum()
    fn = ((y_true==1)&(y_pred==0)).sum()

    precision = tp/(tp+fp+1e-9)
    recall = tp/(tp+fn+1e-9)
    f1 = 2*precision*recall/(precision+recall+1e-9)
    acc = (tp+tn)/(tp+tn+fp+fn+1e-9)
    fpr = fp/(fp+tn+1e-9)

    metrics = dict(acc=acc, precision=precision, recall=recall, f1=f1, fpr=fpr,
                   tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))

    print(f"Isolation Forest结果: Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")
    return metrics

def train_one_class_svm(benign_train, benign_test, malware_test):
    """One-Class SVM基准"""
    print("\n" + "="*70)
    print("训练 One-Class SVM")
    print("="*70)

    train_features = extract_features_batch(benign_train)
    test_benign_features = extract_features_batch(benign_test)
    test_malware_features = extract_features_batch(malware_test)

    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    test_benign_features = scaler.transform(test_benign_features)
    test_malware_features = scaler.transform(test_malware_features)

    clf = OneClassSVM(nu=0.1, kernel='rbf', gamma='auto')
    clf.fit(train_features)

    pred_benign = clf.predict(test_benign_features)
    pred_malware = clf.predict(test_malware_features)

    pred_benign = (pred_benign == -1).astype(int)
    pred_malware = (pred_malware == -1).astype(int)

    y_true = np.array([0]*len(benign_test) + [1]*len(malware_test))
    y_pred = np.concatenate([pred_benign, pred_malware])

    tp = ((y_true==1)&(y_pred==1)).sum()
    tn = ((y_true==0)&(y_pred==0)).sum()
    fp = ((y_true==0)&(y_pred==1)).sum()
    fn = ((y_true==1)&(y_pred==0)).sum()

    precision = tp/(tp+fp+1e-9)
    recall = tp/(tp+fn+1e-9)
    f1 = 2*precision*recall/(precision+recall+1e-9)
    acc = (tp+tn)/(tp+tn+fp+fn+1e-9)
    fpr = fp/(fp+tn+1e-9)

    metrics = dict(acc=acc, precision=precision, recall=recall, f1=f1, fpr=fpr,
                   tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))

    print(f"One-Class SVM结果: Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")
    return metrics

# ============ 【新增】9. 可视化 ============
def visualize_tsne(model, benign_seqs, malware_seqs, window_size=20, device="cuda", save_path="tsne_visualization.png"):
    """t-SNE可视化隐空间分布"""
    print("\n生成t-SNE可视化...")

    model.eval()
    hidden_states = []
    labels = []

    with torch.no_grad():
        for s in benign_seqs[:500]:
            X, _ = make_windows([s], window_size)
            if len(X) == 0:
                continue
            X = torch.tensor(X, dtype=torch.long).to(device)
            _, hidden = model(X, return_hidden=True)
            hidden_states.append(hidden.mean(0).cpu().numpy())
            labels.append(0)

        for s in malware_seqs[:500]:
            X, _ = make_windows([s], window_size)
            if len(X) == 0:
                continue
            X = torch.tensor(X, dtype=torch.long).to(device)
            _, hidden = model(X, return_hidden=True)
            hidden_states.append(hidden.mean(0).cpu().numpy())
            labels.append(1)

    hidden_states = np.array(hidden_states)
    labels = np.array(labels)

    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    embeddings_2d = tsne.fit_transform(hidden_states)

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(
        embeddings_2d[labels==0, 0],
        embeddings_2d[labels==0, 1],
        c='blue', alpha=0.5, label='Benign', s=20
    )
    scatter = plt.scatter(
        embeddings_2d[labels==1, 0],
        embeddings_2d[labels==1, 1],
        c='red', alpha=0.5, label='Malware', s=20
    )
    plt.legend()
    plt.title('t-SNE Visualization of Hidden States')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"可视化已保存到 {save_path}")
    plt.close()

# ============ 10. 主流程 ============
def main():
    path = "./dynamic_api_call_sequence_per_malware_100_0_306.csv"
    seqs, labels = load_csv_expanded(path)
    benign_train, benign_val, benign_test, malware_val, malware_test = split_benign_only(seqs, labels)

    vocab_size = int(seqs.max()) + 1
    print("vocab_size:", vocab_size)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    all_results = {}

    # ========== 实验1：原始Baseline ==========
    print("\n" + "="*70)
    print("实验1: 原始Baseline - 单窗口LSTM")
    print("="*70)

    window_size = 20
    Xtr, ytr = make_windows(benign_train, window_size)
    Xva, yva = make_windows(benign_val, window_size)

    train_loader = DataLoader(WindowDataset(Xtr, ytr), batch_size=256, shuffle=True)
    val_loader = DataLoader(WindowDataset(Xva, yva), batch_size=256)

    model_baseline = LSTMLM(vocab_size, emb_dim=128, hidden_dim=256, num_layers=2, dropout=0.3)
    model_baseline = train_model(model_baseline, train_loader, val_loader, device=device)

    benign_val_scores = sequence_scores_nll(model_baseline, benign_val, window_size, device=device)
    malware_val_scores = sequence_scores_nll(model_baseline, malware_val, window_size, device=device)
    best_th_baseline, _ = search_best_threshold(benign_val_scores, malware_val_scores)

    benign_test_scores = sequence_scores_nll(model_baseline, benign_test, window_size, device=device)
    malware_test_scores = sequence_scores_nll(model_baseline, malware_test, window_size, device=device)
    metrics_baseline = evaluate(best_th_baseline, benign_test_scores, malware_test_scores)

    print(f"\n【Baseline结果】")
    print(f"Threshold: {best_th_baseline:.4f}")
    print(f"Precision: {metrics_baseline['precision']:.4f}, Recall: {metrics_baseline['recall']:.4f}, F1: {metrics_baseline['f1']:.4f}")
    all_results['Baseline (单窗口LSTM)'] = metrics_baseline

    # ========== 实验2：改进版记忆增强LSTM ==========
    print("\n" + "="*70)
    print("实验2: 改进版记忆增强LSTM (余弦相似度 + 大记忆库)")
    print("="*70)

    model_memory = ImprovedMemoryAugmentedLSTM(vocab_size, emb_dim=128, hidden_dim=256,
                                       num_layers=2, dropout=0.3, memory_size=5000)
    model_memory = train_model_with_memory(model_memory, train_loader, val_loader, device=device)

    benign_val_hybrid, alpha_val = sequence_scores_hybrid_improved(model_memory, benign_val, window_size, device=device)
    malware_val_hybrid, _ = sequence_scores_hybrid_improved(model_memory, malware_val, window_size, device=device)
    print(f"自适应权重 alpha (NLL): {alpha_val:.4f}")

    best_th_hybrid, _ = search_best_threshold(benign_val_hybrid, malware_val_hybrid)

    benign_test_hybrid, _ = sequence_scores_hybrid_improved(model_memory, benign_test, window_size, device=device)
    malware_test_hybrid, _ = sequence_scores_hybrid_improved(model_memory, malware_test, window_size, device=device)
    metrics_hybrid = evaluate(best_th_hybrid, benign_test_hybrid, malware_test_hybrid)

    print(f"\n【改进版记忆增强结果】")
    print(f"Precision: {metrics_hybrid['precision']:.4f}, Recall: {metrics_hybrid['recall']:.4f}, F1: {metrics_hybrid['f1']:.4f}")
    all_results['改进版记忆增强'] = metrics_hybrid

    # ========== 实验3：多窗口集成（不带记忆）==========
    print("\n" + "="*70)
    print("实验3: 改进版多窗口集成")
    print("="*70)

    ensemble_basic = ImprovedMultiWindowEnsemble(window_sizes=[10, 20, 30],
                                         vocab_size=vocab_size, use_memory=False)
    ensemble_basic.train_all(benign_train, benign_val, malware_val, device)

    test_seqs = np.concatenate([benign_test, malware_test])
    y_true = np.array([0]*len(benign_test) + [1]*len(malware_test))

    for method in ['vote', 'max', 'weighted']:
        print(f"\n--- Ensemble Method: {method} ---")
        pred_test, _ = ensemble_basic.predict_ensemble(test_seqs, device, method=method)

        tp = ((y_true==1)&(pred_test==1)).sum()
        tn = ((y_true==0)&(pred_test==0)).sum()
        fp = ((y_true==0)&(pred_test==1)).sum()
        fn = ((y_true==1)&(pred_test==0)).sum()

        precision = tp/(tp+fp+1e-9)
        recall = tp/(tp+fn+1e-9)
        f1 = 2*precision*recall/(precision+recall+1e-9)
        acc = (tp+tn)/(tp+tn+fp+fn+1e-9)
        fpr = fp/(fp+tn+1e-9)

        metrics_ensemble = dict(acc=acc, precision=precision, recall=recall, f1=f1, fpr=fpr,
                               tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        all_results[f'多窗口集成-{method}'] = metrics_ensemble

    # ========== 实验4：记忆增强 + 多窗口集成 ==========
    print("\n" + "="*70)
    print("实验4: 改进版记忆增强多窗口集成")
    print("="*70)

    ensemble_memory = ImprovedMultiWindowEnsemble(window_sizes=[10, 20, 30],
                                          vocab_size=vocab_size, use_memory=True)
    ensemble_memory.train_all(benign_train, benign_val, malware_val, device)

    for method in ['vote', 'max', 'weighted']:
        print(f"\n--- Ensemble Method: {method} ---")
        pred_test, _ = ensemble_memory.predict_ensemble(test_seqs, device, method=method)

        tp = ((y_true==1)&(pred_test==1)).sum()
        tn = ((y_true==0)&(pred_test==0)).sum()
        fp = ((y_true==0)&(pred_test==1)).sum()  # 【修正】这里是bug所在
        fn = ((y_true==1)&(pred_test==0)).sum()

        precision = tp/(tp+fp+1e-9)
        recall = tp/(tp+fn+1e-9)
        f1 = 2*precision*recall/(precision+recall+1e-9)
        acc = (tp+tn)/(tp+tn+fp+fn+1e-9)
        fpr = fp/(fp+tn+1e-9)

        metrics_final = dict(acc=acc, precision=precision, recall=recall, f1=f1, fpr=fpr,
                            tp=int(tp), tn=int(tn), fp=int(fp), fn=int(fn))
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        all_results[f'记忆增强集成-{method}'] = metrics_final

    # ========== 实验5-6：传统方法对比 ==========
    metrics_if = train_isolation_forest(benign_train, benign_test, malware_test)
    all_results['Isolation Forest'] = metrics_if

    metrics_svm = train_one_class_svm(benign_train, benign_test, malware_test)
    all_results['One-Class SVM'] = metrics_svm

    # ========== 结果汇总 ==========
    print("\n" + "="*70)
    print("📊 实验结果汇总对比")
    print("="*70)

    print(f"\n{'方法':<30} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'FPR':<12}")
    print("-" * 78)

    for method_name, metrics in all_results.items():
        fpr = metrics.get('fpr', metrics['fp']/(metrics['fp']+metrics['tn']))
        print(f"{method_name:<30} {metrics['precision']:<12.4f} {metrics['recall']:<12.4f} {metrics['f1']:<12.4f} {fpr:<12.4f}")

    best_method = max(all_results.items(), key=lambda x: x[1]['f1'])
    print(f"\n🏆 最佳方法: {best_method[0]}, F1-Score: {best_method[1]['f1']:.4f}")

    baseline_f1 = all_results['Baseline (单窗口LSTM)']['f1']
    print(f"\n📈 相对Baseline的F1提升:")
    for method_name, metrics in all_results.items():
        if method_name != 'Baseline (单窗口LSTM)':
            improvement = (metrics['f1'] - baseline_f1) / baseline_f1 * 100
            print(f"{method_name:<30} {improvement:>6.2f}%")

    # ========== 可视化 ==========
    visualize_tsne(model_memory, benign_test, malware_test, window_size=20, device=device)

    return all_results

if __name__ == "__main__":
    results = main()


vocab_size: 307

实验1: 原始Baseline - 单窗口LSTM


ep1: train=2.3557, val=1.5717


ep5: train=0.7130, val=0.8544


ep10: train=0.4260, val=0.7722


ep15: train=0.2815, val=0.7920


...

ep1: train=2.4429, val=1.6430


ep5: train=0.7367, val=0.8743


ep10: train=0.4260, val=0.7818


ep15: train=0.2818, val=0.7974


Early stop at epoch 16
Val F1: 0.4595, Threshold: 0.3618
Window 10 weight: 0.3370
Window 20 weight: 0.3272
Window 30 weight: 0.3358

--- Ensemble Method: vote ---
Precision: 0.9975, Recall: 0.2979, F1: 0.4588

--- Ensemble Method: max ---
Precision: 0.9967, Recall: 0.5044, F1: 0.6698

--- Ensemble Method: weighted ---
Precision: 0.9970, Recall: 0.4926, F1: 0.6594

训练 Isolation Forest
Isolation Forest结果: Precision=0.9828, Recall=0.0318, F1=0.0616

训练 One-Class SVM
One-Class SVM结果: Precision=0.9969, Recall=0.2045, F1=0.3394

📊 实验结果汇总对比

方法                             Precision    Recall       F1-Score     FPR         
------------------------------------------------------------------------------
Baseline (单窗口LSTM)             0.9986       0.3030       0.4649       0.0691      
改进版记忆增强                        0.9929       0.2931       0.4526       0.3318      
多窗口集成-vote                     0.9985       0.2916       0.4514       0.0691      
多窗口集成-max                      0.9981       0.48